In [ ]:
import pandas as pd
import boto3
from dotenv import load_dotenv
import os
from sqlalchemy import create_engine

In [ ]:
load_dotenv()

ACCESS_KEY_ID = os.getenv('AWS_ACCESS_KEY_ID')
AWS_SECRET_ACCESS_KEY = os.getenv('AWS_SECRET_ACCESS_KEY')
HOSTNAME = os.getenv('HOSTNAME')
USER = os.getenv('USER')
PASSWORD = os.getenv('PASSWORD')
TABLE_NAME = os.getenv('TABLE_NAME')
DATABASE_NAME = os.getenv('DATABASE_NAME')
BUCKET_NAME = os.getenv('BUCKET_NAME')

In [ ]:
#get data
session = boto3.Session(
    aws_access_key_id=ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
)

df1 = pd.read_csv(f's3://{BUCKET_NAME}/bengals.csv')
df2 = pd.read_csv(f's3://{BUCKET_NAME}/boyd_receiving.csv')
df3 = pd.read_csv(f's3://{BUCKET_NAME}/chase_receiving.csv')
df4 = pd.read_csv(f's3://{BUCKET_NAME}/higgins_receiving.csv')

In [ ]:
#transform data
df2.columns = ['Week', 'Boyd Yards', 'Boyd TD']
df3.columns = ['Week', 'Chase Yards', 'Chase TD']
df4.columns = ['Week', 'Higgins Yards', 'Higgins TD']
merge1 = pd.merge(df1, df2, on='Week')
merge2 = pd.merge(merge1, df3, on='Week')
final_merge = pd.merge(merge2, df4, on='Week')
final_merge['Result'] = final_merge['Result'].map({1.0: 'Win', 0: 'Loss'})

In [ ]:
#load to sql server
engine = create_engine(f'postgresql://{USER}:{PASSWORD}@{HOSTNAME}:5432/{DATABASE_NAME}')
final_merge.to_sql(name=TABLE_NAME, con=engine, schema='public', if_exists='replace')


In [ ]:
#dataframe checks
print(df1.head())
print(df2.head())
print(df3.head())
print(df4.head())
print(final_merge.head())